In [26]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import mean_squared_error, root_mean_squared_error

import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/matheusdias/my_stuff/mlops-zoomcamp/cohorts/2024/02-experiment-tracking/notebooks/mlruns/2', creation_time=1716234061540, experiment_id='2', last_update_time=1716234061540, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [17]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [19]:
df_train = read_dataframe('../data/yellow_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/yellow_tripdata_2024-02.parquet')

In [20]:
len(df_train), len(df_val)

(2898906, 2938060)

In [21]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [22]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [23]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [27]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

5.5059366623994395

In [28]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [29]:
with mlflow.start_run():
    mlflow.set_tag("developer", "matheus")

    mlflow.log_param("train-data-path", '../data/yellow_tripdata_2024-01.parquet')
    mlflow.log_param("val-data-path", '../data/yellow_tripdata_2024-02.parquet')

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("model_type", "lasso")
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/Users/matheusdias/my_stuff/mlops-zoomcamp/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [30]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [31]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [34]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status": STATUS_OK}

In [35]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/matheusdias/my_stuff/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [17:32:54] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.70244                           
[1]	validation-rmse:9.20595                           
[2]	validation-rmse:8.75908                           
[3]	validation-rmse:8.35823                           
[4]	validation-rmse:7.99894                           
[5]	validation-rmse:7.67804                           
[6]	validation-rmse:7.39199                           
[7]	validation-rmse:7.13742                           
[8]	validation-rmse:6.91179                           
[9]	validation-rmse:6.71206                           
[10]	validation-rmse:6.53569                          
[11]	validation-rmse:6.37990                          
[12]	validation-rmse:6.24253                          
[13]	validation-rmse:6.12195                          
[14]	validation-rmse:6.01602                          
[15]	validation-rmse:5.92295                          
[16]	validation-rmse:5.84126                          
[17]	validation-rmse:5.76953                          
[18]	valid

/Users/matheusdias/my_stuff/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [17:57:17] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.85494                                                         
[1]	validation-rmse:9.48536                                                         
[2]	validation-rmse:9.14265                                                         
[3]	validation-rmse:8.82522                                                         
[4]	validation-rmse:8.53151                                                         
[5]	validation-rmse:8.26003                                                         
[6]	validation-rmse:8.00975                                                         
[7]	validation-rmse:7.77878                                                         
[8]	validation-rmse:7.56621                                                         
[9]	validation-rmse:7.37109                                                         
[10]	validation-rmse:7.19160                                                        
[11]	validation-rmse:7.02700                                     

...

[0]	validation-rmse:9.77856                                                            
[1]	validation-rmse:9.34309                                                            
[2]	validation-rmse:8.94402                                                            
[3]	validation-rmse:8.57876                                                            
[4]	validation-rmse:8.24515                                                            
[5]	validation-rmse:7.94093                                                            
[6]	validation-rmse:7.66366                                                            
[7]	validation-rmse:7.41193                                                            
[8]	validation-rmse:7.18324                                                            
[9]	validation-rmse:6.97591                                                            
[10]	validation-rmse:6.78867                                                           
[11]	validation-rmse:6.61943    

/Users/matheusdias/my_stuff/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [14:15:31] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.78869                                                          
[1]	validation-rmse:5.49373                                                          
[2]	validation-rmse:5.45629                                                          
[3]	validation-rmse:5.44418                                                          
[4]	validation-rmse:5.43697                                                          
[5]	validation-rmse:5.42983                                                          
[6]	validation-rmse:5.42337                                                          
[7]	validation-rmse:5.41645                                                          
[8]	validation-rmse:5.41076                                                          
[9]	validation-rmse:5.40463                                                          
[10]	validation-rmse:5.39883                                                         
[11]	validation-rmse:5.39354                          

/Users/matheusdias/my_stuff/mlops-zoomcamp/.venv/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [14:33:42] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.75290                                                          
[1]	validation-rmse:6.42641                                                          
[2]	validation-rmse:5.76711                                                          
[3]	validation-rmse:5.44916                                                          
[4]	validation-rmse:5.29416                                                          
[5]	validation-rmse:5.21388                                                          
[6]	validation-rmse:5.16817                                                          
[7]	validation-rmse:5.14043                                                          
[8]	validation-rmse:5.12351                                                          
[9]	validation-rmse:5.11115                                                          
[10]	validation-rmse:5.10007                                                         
[11]	validation-rmse:5.09168                          